---
title: "Código básico para un solo paciente"
format: html
---

In [1]:
import numpy as np
import mne
import yasa
import pandas as pd

from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc

# Información importante

http://dataset.isr.uc.pt/ISRUC_Sleep/Content.pdf

Pacientes Data of Subgroup_3. Cada comprimido tiene:
* un fichero .rec que es un .edf (RENOMBRARLO)
* dos ficheros .txt que son las marcas de los especialistas
* dos fichero .xlsx que contienen la misma información (Stage) y más (muy útil si se desea profundizar en porqué se cometen los errores, para descartar épocas de dudosa calidad, etc.).

https://sleeptight.isr.uc.pt/?page_id=48

El PSG está compuesto por las señales de los siguientes canales:

-   EEG (F3, C3, O1, F4, C4 y O2);
-   EOG, derecho e izquierdo (ROC y LOC);
-   electrocardiograma (ECG);
-   tipos de EMG (un m. submentalis -- EMG de la barbilla (X1) -- y dos m. tibialis -- EMG de las piernas);
-   Las referencias se colocaron en los lóbulos de las orejas izquierda y derecha (A1, A2).


# Canales del polisomnograma

In [7]:
from pathlib import Path

data_path = Path('../dataset/')

In [8]:
path = data_path / '1/1.edf' #camino al fichero del paciente 1
raw = mne.io.read_raw_edf(path, preload=True)
#Eliminamos los canales que no queremos, CUIDADO CON QUITAR MUCHOS CANALES PORQUE DESPUÉS NO HAY INTERPRETACIÓN CLÍNICA
raw.drop_channels(['X4', 'X5', 'X6', 'DC3', 'X7', 'X8', 'SaO2', 'DC8','ROC-A1', 'F3-A2', 'C3-A2', 'O1-A2', 'F4-A1','O2-A1', 'X2', 'X3' ])
#Cambiamos la frecuencia a 100 Hz para reducir el tiempo de cálculo (aprovechamos que no tenemos frecuencias superiores a 50Hz de interés)
raw.resample(100)
#Filtramos la señal para eliminar la línea basal (f muy bajas producidas por la respiración, movimiento de piernas, etc.) 
raw.filter(0.3, 49)
# Observamos los datos
sf = raw.info['sfreq']
chan = raw.ch_names
print('Chan =', chan)
print('Sampling frequency =', sf, 'Hz')
#CUIDADO! si se accede directamente a los datos hay que cambiar la escala!
data = raw.get_data() * 1e6 #microVolts (porque mne trabaja en V)
data = data[:,:-30*30*100] #eliminamos las 30 últimas porque lo indica el artículo de referencia
print('Data shape =', data.shape)


Extracting EDF parameters from c:\Machine Learning\Master Tuebingen\Semester_3\HDA\sleep-stage-detection\dataset\1\1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 5723999  =      0.000 ... 28619.995 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.3 - 49 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Upper passband edge: 49.00 Hz
- Upper transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 49.50 Hz)
- Filter length: 1101 samples (11.010 s)

Chan = ['LOC-A2', 'C4-A1', 'X1']
Sampling frequency = 100.0 Hz
Data shape = (3, 2772000)


# Etiquetas Fases

CODIFICACIÓN DE LAS FASES EN LOS FICHEROS DE ISRUC-SLEEP Dataset (cuidado porque no hay valor 4 (antiguamente se distinguía una fase más))

* TXT->STAGE
* 0->W
* 1->N1
* 2->N2
* 3->N3
* 5->REM

The default hypnogram format in YASA is a 1D integer vector where:
    
* -2 = Unscored
* -1 = Artefact / Movement
* 0 = Wake
* 1 = N1 sleep
* 2 = N2 sleep
* 3 = N3 sleep
* 4 = REM sleep

In [9]:
#Cargamos las etiquetas del primer médico y eliminamos las 30 por que lo indica el artículo 
path_lab= data_path / '1/1_1.txt'  #camino a las etiquetas de marcado del clínico 1
labels1 = pd.read_csv(path_lab, header = None).squeeze("columns")
labels1[labels1==5]=4 #cuidado con la codificación si se quiere utilizar las funciones de YASA... 
labels1 = labels1[:-30]

# Características 

Generamos un registro por cada 30 segundos de polisomnograma. La función SleepStaging realiza esta tarea pero genera las características que considera el autor de la librería. 

In [10]:
#Obtenemos algunas características utilizando la función SleepStaging
sls = yasa.SleepStaging(raw, eeg_name ='C4-A1' ,  eog_name='LOC-A2', emg_name='X1')
#Eliminamos las 30 últimas épocas
sls2 = sls.get_features()[:-30]

sls_train = sls2 
label_train = labels1

For information of created features:
https://raphaelvallat.com/yasa/build/html/generated/yasa.SleepStaging.html

Interesting Comment:
N1 sleep is the sleep stage with the lowest detection accuracy. This is expected because N1 is also the stage with the lowest human inter-rater agreement. Be very careful for potential misclassification of N1 sleep (e.g. scored as Wake or N2) when inspecting the predicted sleep stages.



# Modelo 

Modelo multiclase sin ajuste de parámetros, ni validación ni generalización. 

In [11]:
X_train= sls_train[:]
y_train= label_train[:]

# Binarize the output
y_train = label_binarize(y_train, classes=[0, 1, 2, 3, 4])
n_classes = 5
# Learn to predict each class against the other
classifier = OneVsRestClassifier(
     RandomForestClassifier(n_estimators=1000, criterion="gini", random_state=0)
)
classifier.fit(X_train, y_train)

rf = []
rf.append(classifier)
y_score_train = classifier.predict_proba(X_train)
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
thresholds = dict()
roc_auc_train = dict()
for i in range(n_classes):
    fpr[i], tpr[i], thresholds[i] = roc_curve(y_train[:, i], y_score_train[:, i])
    roc_auc_train[i] = auc(fpr[i], tpr[i])

In [14]:
roc_auc_train

{0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0}

In [17]:
y_train.shape

(924, 5)